# SW-4-SPARQL

**Navigation** : [<< 3-GraphOperations](SW-3-GraphOperations.ipynb) | [Index](README.md) | [5-LinkedData >>](SW-5-LinkedData.ipynb)

## SPARQL : Interroger les graphes RDF

### Duree estimee : 45 minutes

A la fin de ce notebook, vous saurez :
1. Comprendre SPARQL comme le **SQL du Web semantique**
2. Ecrire des requetes **SELECT** avec variables et patterns de triplets
3. Utiliser **FILTER** et **OPTIONAL** pour affiner les resultats
4. Combiner des patterns avec **UNION**
5. Trier et paginer avec **ORDER BY**, **LIMIT**, **OFFSET**
6. Construire des requetes programmatiquement avec le **QueryBuilder** de dotNetRDF

### Prerequis
- .NET 9.0 avec .NET Interactive
- Avoir complete [SW-3-GraphOperations](SW-3-GraphOperations.ipynb)

In [ ]:
#r "nuget: dotNetRDF, 3.2.1"

Installing Packages dotNetRDF

In [ ]:
using VDS.RDF;
using VDS.RDF.Parsing;
using VDS.RDF.Writing;
using VDS.RDF.Query;
using VDS.RDF.Query.Builder;
using System;
using System.IO;
using System.Linq;

Console.WriteLine("dotNetRDF 3.2.1 pret - espaces de noms SPARQL charges.");

dotNetRDF 3.2.1 pret - espaces de noms SPARQL charges.


---

## 1. Introduction a SPARQL

**SPARQL** (SPARQL Protocol and RDF Query Language) est le langage de requete standard du W3C pour interroger des donnees RDF. Il joue pour RDF le meme role que SQL pour les bases relationnelles.

| SQL | SPARQL | Description |
|-----|--------|-------------|
| `SELECT col FROM table WHERE condition` | `SELECT ?var WHERE { pattern }` | Extraire des donnees |
| Tables et colonnes | Graphes et triplets | Structure de donnees |
| `JOIN` | Patterns partageant des variables | Jointure |
| `WHERE condition` | `FILTER(condition)` | Filtrage |

dotNetRDF offre deux approches pour construire des requetes SPARQL :
- **Chaines brutes** : `"SELECT ?x WHERE { ?x ?y ?z }"` -- simple mais pas de validation a la compilation
- **QueryBuilder** : API fluide C# -- type safety, composition dynamique

---

## 2. SELECT : Requetes de base

Une requete SELECT retourne un ensemble de lignes (bindings) pour les variables demandees. Le pattern `WHERE` definit les contraintes sur les triplets.

In [ ]:
// 2.1 Requete SELECT simple avec QueryBuilder
string x = "x";
var queryBuilder = QueryBuilder
    .Select(new string[] { x })
    .Where(
        (triplePatternBuilder) =>
        {
            triplePatternBuilder
                .Subject(x)
                .PredicateUri(new Uri("http://www.w3.org/2001/vcard-rdf/3.0#FN"))
                .Object("John Smith");
        });

var query = queryBuilder.BuildQuery();
Console.WriteLine("=== Requete generee ===");
Console.WriteLine(query.ToString());

=== Requete generee ===


SELECT ?x WHERE
{ ?x <http://www.w3.org/2001/vcard-rdf/3.0#FN> ?John Smith . }



#### Interpretation

La requete generee correspond a :
```sparql
SELECT ?x
WHERE {
    ?x <http://www.w3.org/2001/vcard-rdf/3.0#FN> "John Smith" .
}
```

| Composant | Code QueryBuilder | SPARQL genere |
|-----------|-------------------|---------------|
| Variables retournees | `.Select(new[] { "x" })` | `SELECT ?x` |
| Pattern sujet | `.Subject(x)` | `?x` (variable) |
| Pattern predicat | `.PredicateUri(uri)` | `<uri>` (URI fixe) |
| Pattern objet | `.Object("John Smith")` | `"John Smith"` (litteral) |

In [ ]:
// 2.2 PREFIX avec plusieurs patterns de triplets
var prefixes = new NamespaceMapper(true);
prefixes.AddNamespace("vcard", new Uri("http://www.w3.org/2001/vcard-rdf/3.0#"));

string y = "y";
var givenName = new SparqlVariable("givenName");

var qb = QueryBuilder
    .Select(new SparqlVariable[] { givenName })
    .Where(
        (tp) =>
        {
            tp.Subject(y).PredicateUri("vcard:Family").Object("Smith");
            tp.Subject(y).PredicateUri("vcard:Given").Object(givenName);
        });
qb.Prefixes = prefixes;

Console.WriteLine("=== SELECT avec PREFIX ===");
Console.WriteLine(qb.BuildQuery().ToString());

=== SELECT avec PREFIX ===


PREFIX vcard: <http://www.w3.org/2001/vcard-rdf/3.0#>

SELECT ?givenName WHERE
{ 
  ?y vcard:Family ?Smith . 
  ?y vcard:Given ?givenName . 
}



#### Interpretation

Les **prefixes** abregent les URIs longues :

| Sans prefix | Avec prefix |
|-------------|-------------|
| `<http://www.w3.org/2001/vcard-rdf/3.0#FN>` | `vcard:FN` |

Plusieurs patterns dans le meme `Where()` creent une **conjonction** (AND) : la variable `?y` doit satisfaire les deux contraintes simultanement.

```sparql
PREFIX vcard: <http://www.w3.org/2001/vcard-rdf/3.0#>
SELECT ?givenName
WHERE {
    ?y vcard:Family "Smith" .
    ?y vcard:Given ?givenName .
}
```

---

## 3. FILTER et OPTIONAL

**FILTER** restreint les resultats selon des conditions. **OPTIONAL** permet de recuperer des informations meme si elles n'existent pas.

In [ ]:
// 3.1 FILTER numerique
var prefixes = new NamespaceMapper(true);
prefixes.AddNamespace("info", new Uri("http://somewhere/peopleInfo#"));

string resource = "resource";
string age = "age";

var qb = QueryBuilder
    .Select(new string[] { resource })
    .Where(
        (tp) =>
        {
            tp.Subject(resource).PredicateUri($"info:{age}").Object(age);
        })
    .Filter((b) => b.Variable(age) > 24);
qb.Prefixes = prefixes;

Console.WriteLine("=== FILTER numerique ===");
Console.WriteLine(qb.BuildQuery().ToString());

=== FILTER numerique ===


PREFIX info: <http://somewhere/peopleInfo#>

SELECT ?resource WHERE
{ 
  ?resource info:age ?age . 
  FILTER(?age > 24 ) 
}



In [ ]:
// 3.2 FILTER Regex (expressions regulieres)
var prefixes = new NamespaceMapper(true);
prefixes.AddNamespace("vcard", new Uri("http://www.w3.org/2001/vcard-rdf/3.0#"));

var givenName = new SparqlVariable("givenName");

var qb = QueryBuilder
    .Select(new SparqlVariable[] { givenName })
    .Where(
        (tp) =>
        {
            tp.Subject("y").PredicateUri("vcard:Given").Object(givenName);
        })
    .Filter((b) => b.Regex(b.Variable("givenName"), "sarah", "i"));
qb.Prefixes = prefixes;

Console.WriteLine("=== FILTER Regex ===");
Console.WriteLine(qb.BuildQuery().ToString());

=== FILTER Regex ===


PREFIX vcard: <http://www.w3.org/2001/vcard-rdf/3.0#>

SELECT ?givenName WHERE
{ 
  ?y vcard:Given ?givenName . 
  FILTER(REGEX(?givenName,"sarah","i")) 
}



#### Interpretation : FILTER

| Type de filtre | Syntaxe SPARQL | Code QueryBuilder |
|----------------|----------------|--------------------|
| Comparaison | `FILTER(?age > 24)` | `.Filter(b => b.Variable(age) > 24)` |
| Regex | `FILTER(REGEX(?name, "pattern", "i"))` | `.Filter(b => b.Regex(...))` |
| Egalite | `FILTER(?x = 10)` | `.Filter(b => b.Variable(x) == 10)` |

**Flags Regex** : `i` (insensible a la casse), `m` (multiline), `s` (dot matches newline).

In [ ]:
// 3.3 OPTIONAL avec FILTER
var prefixes = new NamespaceMapper(true);
prefixes.AddNamespace("info", new Uri("http://somewhere/peopleInfo#"));
prefixes.AddNamespace("vcard", new Uri("http://www.w3.org/2001/vcard-rdf/3.0#"));

string name = "name";
string age = "age";
string person = "person";

var qb = QueryBuilder
    .Select(new string[] { name, age })
    .Where(
        (tp) =>
        {
            tp.Subject(person).PredicateUri("vcard:FN").Object(name);
        })
    .Optional(
        (optionalBuilder) =>
        {
            optionalBuilder.Where(
                (tp) =>
                {
                    tp.Subject(person).PredicateUri($"info:{age}").Object(age);
                });
            optionalBuilder.Filter((b) => b.Variable(age) > 42);
        });
qb.Prefixes = prefixes;

Console.WriteLine("=== OPTIONAL avec FILTER ===");
Console.WriteLine(qb.BuildQuery().ToString());

=== OPTIONAL avec FILTER ===


PREFIX info: <http://somewhere/peopleInfo#>
PREFIX vcard: <http://www.w3.org/2001/vcard-rdf/3.0#>

SELECT ?name ?age WHERE
{ 
  ?person vcard:FN ?name . 
  OPTIONAL { 
    ?person info:age ?age . 
    FILTER(?age > 42 ) 
  }
}



#### Interpretation : OPTIONAL

**OPTIONAL** rend un pattern **facultatif** : les resultats sont retournes meme si le pattern optionnel n'a pas de correspondance.

| Sans OPTIONAL | Avec OPTIONAL |
|---------------|---------------|
| Personne sans age : **exclue** | Personne sans age : **incluse** (age = null) |

```sparql
SELECT ?name ?age
WHERE {
    ?person vcard:FN ?name .
    OPTIONAL {
        ?person info:age ?age .
        FILTER(?age > 42)
    }
}
```

Toutes les personnes avec un nom sont retournees ; l'age n'apparait que si > 42.

---

## 4. UNION

**UNION** combine des patterns alternatifs (OR logique) : un resultat est retourne s'il correspond a l'un **ou** l'autre des patterns.

In [ ]:
// 4.1 UNION de deux patterns
var prefixes = new NamespaceMapper(true);
prefixes.AddNamespace("foaf", new Uri("http://xmlns.com/foaf/0.1/"));
prefixes.AddNamespace("vcard", new Uri("http://www.w3.org/2001/vcard-rdf/3.0#"));

string name = "name";
var qb = QueryBuilder.Select(new string[] { name });

qb.Union(
    (unionBuilder) =>
    {
        unionBuilder.Where(
            (tp) => { tp.Subject<IBlankNode>("abc").PredicateUri($"foaf:{name}").Object(name); });
    },
    (unionBuilder) =>
    {
        unionBuilder.Where(
            (tp) => { tp.Subject<IBlankNode>("abc").PredicateUri("vcard:FN").Object(name); });
    });
qb.Prefixes = prefixes;

Console.WriteLine("=== UNION ===");
Console.WriteLine(qb.BuildQuery().ToString());

=== UNION ===


PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX vcard: <http://www.w3.org/2001/vcard-rdf/3.0#>

SELECT ?name WHERE
{ { _:abc foaf:name ?name . } 
  UNION
  { _:abc vcard:FN ?name . } }



#### Interpretation : UNION vs OPTIONAL

```sparql
SELECT ?name
WHERE {
    { _:abc foaf:name ?name }
    UNION
    { _:abc vcard:FN ?name }
}
```

| Cas | UNION | OPTIONAL |
|-----|-------|----------|
| Personne avec foaf:name seulement | Incluse | Incluse |
| Personne avec vcard:FN seulement | Incluse | Depend du pattern principal |
| Personne avec les deux | 2 resultats | 1 resultat |

> **UNION** : alternatives (OR). **OPTIONAL** : complement facultatif.

---

## 5. ORDER BY, LIMIT, OFFSET

Le tri et la pagination controlent l'ordre et la quantite de resultats retournes.

In [ ]:
// 5.1 ORDER BY
var prefixes = new NamespaceMapper(true);
prefixes.AddNamespace("foaf", new Uri("http://xmlns.com/foaf/0.1/"));

string name = "name";
var qb = QueryBuilder
    .Select(new string[] { name })
    .Where(
        (tp) =>
        {
            tp.Subject("x").PredicateUri($"foaf:{name}").Object(name);
        })
    .OrderBy(name);
qb.Prefixes = prefixes;

Console.WriteLine("=== ORDER BY ===");
Console.WriteLine(qb.BuildQuery().ToString());

=== ORDER BY ===


PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?name WHERE
{ ?x foaf:name ?name . }
ORDER BY ASC(?name) 


In [ ]:
// 5.2 LIMIT et OFFSET en chaine brute
string sparql = @"
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?name ?age
WHERE {
    ?person foaf:name ?name .
    ?person foaf:age ?age .
}
ORDER BY DESC(?age)
LIMIT 10
OFFSET 5
";

var sparqlParser = new SparqlQueryParser();
var parsedQuery = sparqlParser.ParseFromString(sparql);

Console.WriteLine("=== LIMIT + OFFSET ===");
Console.WriteLine(parsedQuery.ToString());
Console.WriteLine($"\nType de requete : {parsedQuery.QueryType}");
Console.WriteLine($"Limit           : {parsedQuery.Limit}");
Console.WriteLine($"Offset          : {parsedQuery.Offset}");

=== LIMIT + OFFSET ===


PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?name ?age WHERE
{ 
  ?person foaf:age ?age . 
  ?person foaf:name ?name . 
}
ORDER BY DESC(?age) LIMIT 10 OFFSET 5



Type de requete : Select


Limit           : 10


Offset          : 5


#### Interpretation : Tri et pagination

| Clause | Syntaxe SPARQL | Effet |
|--------|----------------|-------|
| `ORDER BY ?var` | Ascendant (A-Z, 1-9) | Tri des resultats |
| `ORDER BY DESC(?var)` | Descendant (Z-A, 9-1) | Tri inverse |
| `LIMIT n` | Maximum n resultats | Pagination |
| `OFFSET n` | Sauter n premiers resultats | Pagination |

**Tri multiple** : `ORDER BY DESC(?age) ?name` -- trie par age descendant, puis par nom ascendant.

> **SparqlQueryParser** valide la syntaxe a l'execution et expose les proprietes de la requete (`QueryType`, `Limit`, `Offset`).

---

## 6. QueryBuilder : Construction programmatique

Le `QueryBuilder` de dotNetRDF offre une API fluide pour construire des requetes SPARQL dynamiquement, avec validation a la compilation.

In [ ]:
// 6.1 Requete complexe avec QueryBuilder
var prefixes = new NamespaceMapper(true);
prefixes.AddNamespace("foaf", new Uri("http://xmlns.com/foaf/0.1/"));
prefixes.AddNamespace("info", new Uri("http://somewhere/peopleInfo#"));

string person = "person";
string name = "name";
string age = "age";
string email = "email";

var qb = QueryBuilder
    .Select(new string[] { name, age, email })
    .Where(
        (tp) =>
        {
            tp.Subject(person).PredicateUri($"foaf:{name}").Object(name);
            tp.Subject(person).PredicateUri($"info:{age}").Object(age);
        })
    .Optional(
        (opt) =>
        {
            opt.Where(
                (tp) =>
                {
                    tp.Subject(person).PredicateUri("foaf:mbox").Object(email);
                });
        })
    .Filter((b) => b.Variable(age) > 18)
    .OrderBy(name);
qb.Prefixes = prefixes;

Console.WriteLine("=== Requete complexe ===");
Console.WriteLine(qb.BuildQuery().ToString());

=== Requete complexe ===


PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX info: <http://somewhere/peopleInfo#>

SELECT ?name ?age ?email WHERE
{ 
  ?person foaf:name ?name . 
  ?person info:age ?age . 
  OPTIONAL { ?person foaf:mbox ?email . } 
  FILTER(?age > 18 ) 
}
ORDER BY ASC(?name) 


In [ ]:
// 6.2 Execution sur un graphe local
IGraph g = new Graph();
string ttlData = @"
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix ex: <http://example.org/> .

ex:alice foaf:name ""Alice"" ;
         foaf:age 30 ;
         foaf:mbox <mailto:alice@example.org> .

ex:bob   foaf:name ""Bob"" ;
         foaf:age 25 .

ex:charlie foaf:name ""Charlie"" ;
           foaf:age 35 ;
           foaf:mbox <mailto:charlie@example.org> .

ex:diana foaf:name ""Diana"" ;
         foaf:age 17 .
";

new TurtleParser().Load(g, new StringReader(ttlData));
Console.WriteLine($"Graphe de test charge : {g.Triples.Count} triplets");

string sparql = @"
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?name ?age
WHERE {
    ?person foaf:name ?name .
    ?person foaf:age ?age .
    FILTER(?age >= 18)
}
ORDER BY ?name
";

var results = g.ExecuteQuery(sparql) as SparqlResultSet;
Console.WriteLine($"\n{results.Count} resultats :");
foreach (var result in results)
{
    Console.WriteLine($"  {result["name"]} - age: {result["age"]}");
}

Graphe de test charge : 10 triplets



3 resultats :


  Alice^^http://www.w3.org/2001/XMLSchema#string - age: 30^^http://www.w3.org/2001/XMLSchema#integer


  Bob^^http://www.w3.org/2001/XMLSchema#string - age: 25^^http://www.w3.org/2001/XMLSchema#integer


  Charlie^^http://www.w3.org/2001/XMLSchema#string - age: 35^^http://www.w3.org/2001/XMLSchema#integer


#### Interpretation : Execution locale

dotNetRDF inclut un moteur SPARQL complet qui peut executer des requetes directement sur un `IGraph` en memoire.

| Methode | Usage |
|---------|-------|
| `g.ExecuteQuery(sparql)` | Execution sur un graphe local |
| `SparqlResultSet` | Resultat de type SELECT (table de bindings) |
| `IGraph` | Resultat de type CONSTRUCT/DESCRIBE (graphe) |
| `result["name"]` | Acces a une variable du binding |

> Le moteur SPARQL local supporte la majorite de SPARQL 1.1 : SELECT, CONSTRUCT, ASK, DESCRIBE, GROUP BY, HAVING, etc.

In [ ]:
// 6.3 Requete OPTIONAL executee sur le graphe local
string sparqlOptional = @"
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?name ?email
WHERE {
    ?person foaf:name ?name .
    OPTIONAL { ?person foaf:mbox ?email . }
}
ORDER BY ?name
";

var results = g.ExecuteQuery(sparqlOptional) as SparqlResultSet;
Console.WriteLine($"{results.Count} resultats (avec OPTIONAL email) :");
foreach (var result in results)
{
    string emailStr = result.HasBoundValue("email") ? result["email"].ToString() : "(non renseigne)";
    Console.WriteLine($"  {result["name"]} - email: {emailStr}");
}

4 resultats (avec OPTIONAL email) :


  Alice^^http://www.w3.org/2001/XMLSchema#string - email: mailto:alice@example.org


  Bob^^http://www.w3.org/2001/XMLSchema#string - email: (non renseigne)


  Charlie^^http://www.w3.org/2001/XMLSchema#string - email: mailto:charlie@example.org


  Diana^^http://www.w3.org/2001/XMLSchema#string - email: (non renseigne)


#### Interpretation

La requete retourne les 4 personnes. Alice et Charlie ont un email, Bob et Diana non. Grace a `OPTIONAL`, les personnes sans email sont quand meme incluses dans les resultats.

La methode `result.HasBoundValue("email")` permet de tester si une variable optionnelle a ete liee.

---

## 7. Exercices pratiques

### Exercice 1 : SELECT avec FILTER

Ecrivez une requete SPARQL qui selectionne les noms et ages des personnes de plus de 20 ans a partir du graphe de test. Executez-la avec `g.ExecuteQuery()`.

In [ ]:
// Exercice 1 : Votre code ici
// string sparql = @"PREFIX foaf: ...
// SELECT ?name ?age WHERE { ... FILTER(?age > 20) } ORDER BY DESC(?age)";
// var results = g.ExecuteQuery(sparql) as SparqlResultSet;
// foreach (var r in results) Console.WriteLine(...);

### Exercice 2 : QueryBuilder avec OPTIONAL

Utilisez le `QueryBuilder` pour construire une requete qui retourne le nom et l'email optionnel de toutes les personnes, triee par nom.

In [ ]:
// Exercice 2 : Votre code ici
// var prefixes = new NamespaceMapper(true);
// prefixes.AddNamespace("foaf", ...);
// var qb = QueryBuilder.Select(...).Where(...).Optional(...).OrderBy(...);
// qb.Prefixes = prefixes;
// Console.WriteLine(qb.BuildQuery().ToString());

### Exercice 3 : Requete sur animals.ttl

Chargez `data/animals.ttl` et ecrivez une requete SPARQL qui retourne le nom et l'age de tous les animaux de type `ex:Dog`.

In [ ]:
// Exercice 3 : Votre code ici
// IGraph animals = new Graph();
// new TurtleParser().Load(animals, "data/animals.ttl");
// string sparql = @"PREFIX ex: <http://example.org/animals#>
// SELECT ?name ?age WHERE {
//     ?animal a ex:Dog . ?animal ex:name ?name . ?animal ex:age ?age .
// }";
// var results = animals.ExecuteQuery(sparql) as SparqlResultSet;

---

## Resume

| Clause SPARQL | Methode QueryBuilder | Fonction |
|---------------|----------------------|----------|
| `SELECT ?var` | `.Select(variables)` | Variables a retourner |
| `WHERE { pattern }` | `.Where(triplePattern)` | Patterns de triplets |
| `FILTER(cond)` | `.Filter(condition)` | Conditions de filtrage |
| `OPTIONAL { }` | `.Optional(pattern)` | Patterns facultatifs |
| `UNION { } { }` | `.Union(pattern1, pattern2)` | Alternatives (OR) |
| `ORDER BY ?var` | `.OrderBy(var)` | Tri des resultats |
| `LIMIT n` | (chaine brute) | Limiter le nombre de resultats |
| `OFFSET n` | (chaine brute) | Sauter les premiers resultats |

| Approche | Avantage | Cas d'usage |
|----------|----------|-------------|
| **Chaine brute** | Simple, LIMIT/OFFSET direct | Requetes fixes, connues a l'avance |
| **QueryBuilder** | Type safety, composition | Requetes dynamiques, parametrees |
| **SparqlQueryParser** | Validation de syntaxe | Verification avant execution |

### Prochaine etape

Dans **SW-5-LinkedData**, nous apprendrons a interroger des endpoints SPARQL distants comme DBpedia et Wikidata.

---

**Navigation** : [<< 3-GraphOperations](SW-3-GraphOperations.ipynb) | [Index](README.md) | [5-LinkedData >>](SW-5-LinkedData.ipynb)